In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

sub_folder = 'data'

In [2]:
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque

In [3]:
#pip install dash

In [4]:
columns_servidores = ['tiempo','servidor','path', 'metrica', 'value', 'muestreo']

name_cms = 'data/AplicacionCMS'
name_crm = 'data/AplicacionCRM-MOVIL'
name_trb = 'data/AplicativoTRB'

In [5]:
def read_servidores(csv_name):
    data = pd.read_csv(csv_name, sep = ',', header = None)
    data.columns = columns_servidores
    
    data['tiempo'] = pd.to_datetime(data['tiempo'])
    data = data.sort_values(by=['tiempo'])

    data.set_index('tiempo', inplace=True)
    return data


def read_incidencias(excel_name):
    data = pd.read_excel(excel_name)
    
    data['tiempo'] = data['Fecha'].astype(str) + ' ' + data['HORA INICIO'].astype(str)
    
    data['tiempo'] = pd.to_datetime(
        data['tiempo'],errors='coerce',format='%Y/%m/%d %H:%M:%S'
    )
    
    data = data.sort_values(by=['tiempo'])
    data.set_index('tiempo', inplace=True)
    
    return data

def read_incidencias2(excel_name):
    data = pd.read_excel(excel_name)
    
    data['tiempo'] = data['FECHA DE INICIO'].astype(str) + ' ' + data['HORA DE INICIO'].astype(str)
    
    data['tiempo'] = pd.to_datetime(
        data['tiempo'],errors='coerce',format='%Y/%m/%d %H:%M:%S'
    )
    
    data = data.sort_values(by=['tiempo'])
    data.set_index('tiempo', inplace=True)
    
    return data

## MCSS

In [6]:
#mcss_servidores = read_servidores(name_mcss + 'MCSS - Servidores - AGO2019_ENE2020.csv')

In [7]:
#mcss_incidencias = read_incidencias(name_mcss + 'MCSS - Datos de Servicio completo.xlsx')

In [8]:
#mcss_servidores_group = mcss_servidores.groupby(by=['servidor', 'metrica', 'path'])

# CRM

In [9]:
crm_servidores = read_servidores(
    os.path.join(
    os.path.abspath(os.getcwd()),
    'CRM-MOVIL/CRM - Servidores - AGO2019_ENE2020.csv'
)
)

In [10]:
crm_incidencias = read_incidencias(
    os.path.join(
    os.path.abspath(os.getcwd()),
    'CRM-MOVIL/CRM - Datos de Servicio completo.xlsx'
)
)

In [38]:
crm_servidores_group = crm_servidores.groupby(by=['servidor', 'metrica', 'path'])

In [159]:
#incidencias = crm_incidencias

## CMS

In [11]:
#CMS_servidores = read_servidores('data/otro1/CMS - INC00000003265934 - Servidores involucrados CMS.csv') 

In [165]:
#CMS_incidencias = read_incidencias2('data/incidencias/Registro de INCIDENCIAS - 6 MESES.xlsx')

In [39]:
#CMS_servidores_group = CMS_servidores.groupby(by=['servidor', 'metrica', 'path'])

In [172]:
#CMS_servidores_group.first()

## obtener por servidor

### CRM

In [40]:
prd11crm_cpu = crm_servidores_group.get_group(('prd11crm','QOS_CPU_USAGE','prd11crm')).copy()
prd11crm_mem = crm_servidores_group.get_group(('prd11crm','QOS_MEMORY_PERC_USAGE','prd11crm')).copy()


prd19crm_cpu = crm_servidores_group.get_group(('prd19crm','QOS_CPU_USAGE','prd19crm')).copy()
prd19crm_mem = crm_servidores_group.get_group(('prd19crm','QOS_MEMORY_PERC_USAGE','prd19crm')).copy()

In [41]:
names_servidores = ['prd11crm', 'prd19crm']
list_cpus = [prd11crm_cpu, prd19crm_cpu]
list_mems = [prd11crm_mem, prd19crm_mem]

incidencias = crm_incidencias.copy()
incidencias['value'] = 1000

### CMS

In [173]:
"""
sun6c52_cpu = CMS_servidores_group.get_group(('sun6c52','QOS_CPU_USAGE','sun6c52')).copy()
sun6c52_mem = CMS_servidores_group.get_group(('sun6c52','QOS_MEMORY_PERC_USAGE','sun6c52')).copy()

SUN6C57_cpu = CMS_servidores_group.get_group(('SUN6C57','QOS_CPU_USAGE','sun6c57')).copy()
SUN6C57_mem = CMS_servidores_group.get_group(('SUN6C57','QOS_MEMORY_PERC_USAGE','sun6c57')).copy()
"""

In [42]:
"""
names_servidores = ['sun6c52', 'SUN6C57']
list_cpus = [sun6c52_cpu, SUN6C57_cpu]
list_mems = [sun6c52_mem, SUN6C57_mem]

incidencias = CMS_incidencias[CMS_incidencias['SERVIDOR']=='sun6c52'].copy()
incidencias['value'] = 1000
"""

"\nnames_servidores = ['sun6c52', 'SUN6C57']\nlist_cpus = [sun6c52_cpu, SUN6C57_cpu]\nlist_mems = [sun6c52_mem, SUN6C57_mem]\n\nincidencias = CMS_incidencias[CMS_incidencias['SERVIDOR']=='sun6c52'].copy()\nincidencias['value'] = 1000\n"

### MCSS

In [184]:
prd03slr_cpu = mcss_servidores_group.get_group(('prd03slr','QOS_CPU_USAGE','prd03slr')).copy()
prd03slr_mem = mcss_servidores_group.get_group(('prd03slr','QOS_MEMORY_PERC_USAGE','prd03slr')).copy()

prd20mcs_cpu = mcss_servidores_group.get_group(('prd20mcs','QOS_CPU_USAGE','prd20mcs')).copy()
prd20mcs_mem = mcss_servidores_group.get_group(('prd20mcs','QOS_MEMORY_PERC_USAGE','prd20mcs')).copy()

In [185]:
names_servidores = ['prd03slr', 'prd20mcs']
list_cpus = [prd03slr_cpu, prd20mcs_cpu]
list_mems = [prd03slr_mem, prd20mcs_mem]

incidencias = mcss_incidencias.copy()
incidencias['value'] = 1000

# Anomalias

In [44]:
pip install adtk

     |████████████████████████████████| 61kB 210kB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:01
  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23274 sha256=0349696afa71beec7e04d80bb8942f388806a33907591feb684b1b000b3be119
  Stored in directory: /home/villacorta/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [43]:
from adtk.detector import LevelShiftAD
from adtk.visualization import plot

ModuleNotFoundError: No module named 'adtk'

In [187]:
level_shift_ad = LevelShiftAD(c=6.0, side='both', window=5)
anomalies_cpus = []
anomalies_mems = []
for _server in list_cpus:
    anomalies = level_shift_ad.fit_detect(_server['value'])
    anomalies_cpus.append(anomalies)
    
for _server in list_mems:
    anomalies = level_shift_ad.fit_detect(_server['value'])
    anomalies_mems.append(anomalies)

In [188]:
level_shift_ad = LevelShiftAD(c=6.0, side='both', window=5)
anomalies = level_shift_ad.fit_detect(list_cpus[0]['value'])

In [189]:
%matplotlib qt
plot(list_cpus[0]['value'], anomaly_pred=anomalies, ts_linewidth=1, ts_markersize=3, ap_color='red');

# Dashboard

In [190]:
colors = [(150, 150, 150), (8, 154, 255),(0, 253, 255),(231, 217, 0),(0, 0, 190),(0, 184, 113), (98, 24, 91)]
colors = [tuple(reversed(i)) for i in colors]

start_cpu = 0
start_mem = 0
start_anom = 0
STEP = 100
index_cpu = STEP
index_mem = STEP
index_anom = STEP

In [191]:
app = dash.Dash(__name__)
app.layout = html.Div([html.Div([dcc.Graph(id='cpu', style={'width': '75%', 'display': 'inline-block'} ),
                                 dcc.Interval(id='update-cpu',
                                              interval=1*1000)],
                                style={'float': 'left', 'width': '100%', 'text-align': 'center'},),
                       
                       html.Div([dcc.Graph(id='mem', style={'width': '75%', 'display': 'inline-block'} ),
                                 dcc.Interval(id='update-mem',
                                              interval=1*1000)],
                                style={'float': 'left', 'width': '100%', 'text-align': 'center'},),
                       
                       html.Div([dcc.Graph(id='anomalias', style={'width': '75%', 'display': 'inline-block'} ),
                                 dcc.Interval(id='update-anomalias',
                                              interval=1*1000)],
                                style={'float': 'left', 'width': '100%', 'text-align': 'center'},),
                       
                       html.Div([dcc.Graph(id='medidor', style={'width': '75%', 'display': 'inline-block'} ),
                                 dcc.Interval(id='update-medidor',
                                              interval=1*1000)],
                                style={'float': 'left', 'width': '50%', 'text-align': 'center'},),
                       
                      ])

In [192]:
@app.callback(Output('cpu', 'figure'),
              [Input('update-cpu', 'interval')])
def update_cpu():
    global index_cpu
    global start_cpu
    try:
        # we show 4 day approx
        if index_cpu>500: start_cpu += STEP
        
        data_list = []
        
        for i,_server in enumerate(list_cpus):
            #X = cpu_s1.index[0:index_cpu]
            X = _server.index[start_cpu:index_cpu]
            Y = _server['value'].values[start_cpu:index_cpu]
            data = plotly.graph_objs.Scatter(x=X, y=Y, showlegend=True,
                                             name=names_servidores[i], mode='lines',
                                             line=dict(color='rgb({},{},{})'.format(*colors[i])))
            data_list.append(data)
                   
        index_cpu += STEP
        
        time_min = X.min()
        time_max = X.max()
        
        return {'data': data_list, 'layout' : dict(xaxis={'title': 'tiempo','range':[time_min, time_max]},
                                                   yaxis={'title': 'valor', 'range': [0, 100]},
                                                   legend={'x': 0, 'y': 1},
                                                   title='Monitoreo de CPU',
                                                    )}
    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))


@app.callback(Output('mem', 'figure'),
              [Input('update-mem', 'interval')])
def update_mem():
    global index_mem
    global start_mem
    try:
        # we show 4 day approx
        if index_mem>500: start_mem += STEP
            
        data_list = []
        
        for i,_server in enumerate(list_mems):
            X = _server.index[start_mem:index_mem]
            Y = _server['value'].values[start_mem:index_mem]
            data = plotly.graph_objs.Scatter(x=X, y=Y, showlegend=True,
                                             name=names_servidores[i], mode='lines',
                                             line=dict(color='rgb({},{},{})'.format(*colors[i])))
            data_list.append(data)
                    
        index_mem += STEP
        
        time_min = X.min()
        time_max = X.max()
        
        
        return {'data': data_list,'layout': dict(xaxis={'title': 'tiempo','range':[time_min, time_max]},
                                                   yaxis={'title': 'valor', 'range': [0, 100]},
                                                   legend={'x': 0, 'y': 1},
                                                   title='Monitoreo de Memoria',
                                                    )}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))


@app.callback(Output('anomalias', 'figure'),
              [Input('update-anomalias', 'interval')])
def update_anomalias():
    global index_anom
    global start_anom
    try:
        # we show 4 day approx
        if index_anom>500: start_anom += STEP
            
        X = anomalies_cpus[0].index[start_anom:index_anom]
        data_list = []
        
        for _server in anomalies_cpus:
            data = dict(
                x=_server.index[start_anom:index_anom],
                #y=incidencias['value'],
                y=_server.values[start_anom:index_anom],
                #text=df_by_continent['country'],
                showlegend=True,
                mode='markers',
                opacity=0.7,
                marker={
                    'size': 15,
                    'line': {'width': 0.5, 'color': 'white'}
                },
                name='anomalias cpu'
                )
            
            data_list.append(data)
            
        for _server in anomalies_mems:
            data = dict(
                x=_server.index[start_anom:index_anom],
                y=_server.values[start_anom:index_anom],
                showlegend=True,
                mode='markers',
                opacity=0.7,
                marker={
                    'size': 15,
                    'line': {'width': 0.5, 'color': 'white'}
                },
                name='anomalias mem'
                )
            
            data_list.append(data)
            
        '''
        data = dict(
            x=anomalies_mems[1].index[0:index_anom],
            #y=incidencias['value'],
            y=anomalies_mems[1].values[0:index_anom],
            #text=df_by_continent['country'],
            showlegend=True,
            mode='markers',
            opacity=0.7,
            marker={
                'size': 15,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name='anomalias'
        )
        
        data_list.append(data)
        '''
        data = plotly.graph_objs.Bar(
                x = incidencias.index,
                y = incidencias['value'],
                textposition = 'outside')
        data_list.append(data)
        
        index_anom += STEP

        time_min = X.min()
        time_max = X.max()
        
        return {'data': data_list,'layout': dict(xaxis={'title': 'tiempo','range':[time_min, time_max]},
                                                   yaxis={'title': 'Detección', 'range': [0.5, 1.5]},
                                                   legend={'x': 0, 'y': 1},
                                                   title='Anomalias',
                                                    )}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            
@app.callback(Output('medidor', 'figure'),
              [Input('update-medidor', 'interval')])
def update_medidor():
    try:
        labels = ['ninguno', 'bajo','medio','alto']
        Y = [0,0,1,0]
        data = {'x': labels,'y': Y, 'type': 'bar', 'name': 'SF'}
        data = plotly.graph_objs.Indicator(
                        mode = "gauge+number",
                        value = 270,
                        domain = {'x': [0, 1], 'y': [0, 1]},
                        title = {'text': "Speed"})
        return {'data':[data]}
        #return {'data': [data], 'layout': {'title': 'Medidor',
         #                                  'xaxis': {'title':'nivel', 
          #                                           'zeroline': False,
           #                                          'showgrid': False},
            #                               
             #                       }
              # }
        
        '''
        data = plotly.graph_objs.Bar(
                x = Labels,
                y = Y,
                textposition = 'outside'
                )
        
        data = dict(
            x=incidencias.index,
            y=incidencias['value'],
            #text=df_by_continent['country'],
            mode='markers',
            opacity=0.7,
            marker={
                'size': 15,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name='anomalias'
        )
           
        return {'data': [data],'layout': dict(xaxis={'title': 'nivel de anomalías'},
                                                   yaxis={'range': [0, 10]},
                                                   legend={'x': 0, 'y': 1},
                                                   title='Anomalias',
                                                    )}
        '''
    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))

In [193]:
app.scripts.config.serve_locally = True
app.css.config.serve_locally = True
app.run_server(host='localhost')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Feb/2020 13:59:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 13:59:06] "GET /_favicon.ico HTTP/1.1" 